In [ ]:
#q1

In [1]:
import seaborn as sns
df=sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [2]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [3]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [4]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [5]:
df.time

0      Dinner
1      Dinner
2      Dinner
3      Dinner
4      Dinner
        ...  
239    Dinner
240    Dinner
241    Dinner
242    Dinner
243    Dinner
Name: time, Length: 244, dtype: category
Categories (2, object): ['Lunch', 'Dinner']

In [6]:
from sklearn.preprocessing import LabelEncoder

In [7]:
encoder=LabelEncoder()

In [8]:
df['time']=encoder.fit_transform(df['time'])

In [9]:
df.time.unique()

array([0, 1])

In [10]:
X=df.drop(labels=['time'],axis=1)
y=df.time

In [11]:
X.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [12]:
X['day'].value_counts()

Sat     87
Sun     76
Thur    62
Fri     19
Name: day, dtype: int64

In [23]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer ## handling misssing values
from sklearn.preprocessing import StandardScaler ## Feature scaling
from sklearn.preprocessing import OneHotEncoder## categorical to numerical
from sklearn.compose import ColumnTransformer

In [25]:
categorical_cols=['sex','smoker','day']
numerical_cols=['total_bill','tip','size']

In [26]:
## feature Engineering Automation
## Numerical Pipelines
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler()) 
    ]
    
     
 
)

# categorical Pipeline
cat_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('Onehotencoder',OneHotEncoder())
        ]
)
    

In [27]:
preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
])

In [34]:
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [40]:
## Model training automation
models={
    'Random Forest':RandomForestClassifier(),
    'Logistic Regression':LogisticRegression(),
    'Decision Tree':DecisionTreeClassifier()

}

In [41]:
from sklearn.metrics import accuracy_score

In [42]:
def evaluate_model(X_train,y_train,X_test,y_test,models):
    
    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        # Train model
        model.fit(X_train,y_train)

            

        # Predict Testing data
        y_test_pred =model.predict(X_test)

        # Get accuracy for test data prediction
       
        test_model_score = accuracy_score(y_test,y_test_pred)

        report[list(models.keys())[i]] =  test_model_score
            

            
    return report


In [43]:
evaluate_model(X_train,y_train,X_test,y_test,models)

{'Random Forest': 0.9591836734693877,
 'Logistic Regression': 1.0,
 'Decision Tree': 0.9387755102040817}

In [47]:
classifier=RandomForestClassifier()

In [48]:
# hyperparsmeter tunining
params={'max_depth':[3,5,10,None],
        'n_estimators':[100,200,300],
        'criterion':['gini','entropy']
}

In [49]:
from sklearn.model_selection import RandomizedSearchCV

In [50]:
cv=RandomizedSearchCV(classifier,param_distributions=params,scoring='accuracy',cv=5,verbose=3)
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=gini, max_depth=None, n_estimators=300;, score=0.974 total time=   0.4s
[CV 2/5] END criterion=gini, max_depth=None, n_estimators=300;, score=0.923 total time=   0.4s
[CV 3/5] END criterion=gini, max_depth=None, n_estimators=300;, score=1.000 total time=   0.4s
[CV 4/5] END criterion=gini, max_depth=None, n_estimators=300;, score=0.949 total time=   0.4s
[CV 5/5] END criterion=gini, max_depth=None, n_estimators=300;, score=0.923 total time=   0.4s
[CV 1/5] END criterion=gini, max_depth=None, n_estimators=200;, score=0.974 total time=   0.3s
[CV 2/5] END criterion=gini, max_depth=None, n_estimators=200;, score=0.923 total time=   0.3s
[CV 3/5] END criterion=gini, max_depth=None, n_estimators=200;, score=1.000 total time=   0.3s
[CV 4/5] END criterion=gini, max_depth=None, n_estimators=200;, score=0.949 total time=   0.3s
[CV 5/5] END criterion=gini, max_depth=None, n_estimators=200;, score=0.949 total ti

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [51]:
cv.best_params_

{'n_estimators': 200, 'max_depth': None, 'criterion': 'gini'}

In [52]:
#q2

In [53]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

iris = load_iris()
X = iris.data
y = iris.target


In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=45)


In [55]:
rfc = RandomForestClassifier(random_state=42)
lr = LogisticRegression(random_state=42)
voting_clf = VotingClassifier(estimators=[('rfc', rfc), ('lr', lr)], voting='hard')


In [ ]:
# We are using the hard voting method, which means the predicted class is the mode of the predicted classes by each classifier.

In [56]:
pipeline = Pipeline([
    ('voting_clf', voting_clf)
])

pipeline.fit(X_train, y_train)


Pipeline(steps=[('voting_clf',
                 VotingClassifier(estimators=[('rfc',
                                               RandomForestClassifier(random_state=42)),
                                              ('lr',
                                               LogisticRegression(random_state=42))]))])

In [57]:
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)


Accuracy:  0.9733333333333334
